# imports

In [14]:
import requests
import pandas as pd
from datetime import datetime

# acquire data

In [2]:
# note that this had to strip the last two parameters from the api url in the google doc
# also upped the limit 
api_url = "https://opendata.maryland.gov/api/id/crti-ybyp.json?$select=*&$order=`:id`+ASC&$limit=3000&$offset=0"
response = requests.get(api_url)
data = response.json()

# IF THE INTERNET SUXX
# import json
# with open("local_data_copy.json", "r") as input_file:
#     data = json.load(input_file)

In [3]:
data[0:2]

[{'sno': '1',
  'complaint': '232264',
  'complaint_description': 'concern of loud noise in the Cheverly area',
  'complaint_type': 'Other',
  'recieved_date': '2024-03-06',
  'incident_date': '2024-03-06',
  'county': "Prince George's",
  'incident_closed_date': '2024-03-08',
  'incident_status_desc': 'Incident Closed-Managed'},
 {'sno': '2',
  'complaint': '232263',
  'complaint_description': 'concern of loud noise in the Cheverly area',
  'complaint_type': 'Other',
  'recieved_date': '2024-03-06',
  'incident_date': '2024-03-06',
  'county': "Prince George's",
  'incident_closed_date': '2024-03-08',
  'incident_status_desc': 'Incident Closed-Managed'}]

In [5]:
df = pd.DataFrame(data)

In [6]:
df

,sno,complaint,complaint_description,complaint_type,recieved_date,incident_date,county,incident_closed_date,incident_status_desc,incident_zip
0,1,232264,concern of loud noise in the Cheverly area,Other,2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
1,2,232263,concern of loud noise in the Cheverly area,Other,2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
2,3,232262,concern of loud noise in the Cheverly area,Other,2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
3,4,232261,concern of loud noise in the Cheverly area,Other,2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
4,5,232260,concern of loud noise in the Cheverly area,Other,2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
...,...,...,...,...,...,...,...,...,...,...
1514,1515,8058,Boat company sandblasting boats and blowing wh...,Fugitive Dust/Particulate Matter,2021-01-11,2021-01-11,Queen Anne's,2022-03-08,Incident Closed - No further action,NaN
1515,1516,8057,Smoke and ash from neighbor's open burning in ...,Air,2021-01-07,2021-01-07,Anne Arundel,2021-01-07,Incident Closed-No Violation Observed,20751
1516,1517,8056,Smoke from neighbor's wood burning stove.,Air,2021-01-07,2021-01-04,Charles,NaN,Under Investigation,NaN
1517,1518,8051,Concern of materials being burned in fireplace...,Air,2021-01-04,2020-12-28,Frederick,2021-01-15,Incident Closed-No Violation Observed,21770


# clean data

In [7]:
# Before we do anything else, let's create a backup dataframe that we'll never manipulate.
backup_df = df.copy()

In [11]:
# # Let's look at the data types for each column.
#df.dtypes

# That's a lot of "Objects" (meaning strings)! Let's convert the 'received date' to actual dates.
df['recieved_date'] = pd.to_datetime(df['recieved_date'])

# While we're at it, how about we do the same for the two other dates?
df['incident_date'] = pd.to_datetime(df['incident_date'])
df['incident_closed_date'] = pd.to_datetime(df['incident_closed_date'])

In [12]:
df.dtypes

sno                              object
complaint                        object
complaint_description            object
complaint_type                   object
recieved_date            datetime64[ns]
incident_date            datetime64[ns]
county                           object
incident_closed_date     datetime64[ns]
incident_status_desc             object
incident_zip                     object
dtype: object

# analyze data

In [32]:
# For the summaries we're running, we'll limit that to just complaints received in the last year?
#year_ago_date = pd.Timestamp('2023-03-09')
today_date = pd.Timestamp.today()
year_ago_date = today_date - pd.DateOffset(years=1)

last_year_df = df[df['recieved_date'] > year_ago_date].copy()

last_year_df['county'].value_counts()

county
Baltimore City        101
Anne Arundel           85
Prince George's        83
Frederick              62
Baltimore              57
Montgomery             35
Cecil                  25
Dorchester             17
Harford                17
Howard                 15
Allegany               15
Washington             14
Charles                12
Carroll                 9
Wicomico                8
Not Yet Determined      7
Worcester               7
St. Mary's              6
Garrett                 5
Caroline                3
Queen Anne's            2
Somerset                2
Talbot                  1
Kent                    1
Statewide               1
Name: count, dtype: int64

In [33]:
last_year_df['complaint_type'].value_counts()

complaint_type
Air                                                                                 166
Odor                                                                                140
Other                                                                               121
Fugitive Dust/Particulate Matter                                                     35
Smoke                                                                                28
Air, Fumes, Odor                                                                     20
Open Burning                                                                         13
Air, Odor                                                                            11
Fumes                                                                                11
Open Burning, Smoke                                                                   7
Air, Odor, Smoke                                                                      4
Air, Fugitive Dus

In [34]:
last_year_df['complaint_type'] = last_year_df['complaint_type'].str.split(', ')

## Using .explode() to split rows with multiple entities in complaint_type column

In [42]:
# Look at the first five rows.
last_year_df[0:19]

,sno,complaint,complaint_description,complaint_type,recieved_date,incident_date,county,incident_closed_date,incident_status_desc,incident_zip
0,1,232264,concern of loud noise in the Cheverly area,[Other],2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
1,2,232263,concern of loud noise in the Cheverly area,[Other],2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
2,3,232262,concern of loud noise in the Cheverly area,[Other],2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
3,4,232261,concern of loud noise in the Cheverly area,[Other],2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
4,5,232260,concern of loud noise in the Cheverly area,[Other],2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
5,6,232259,concern of loud noise in the Cheverly area,[Other],2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
6,7,232258,concern of loud noise in the Cheverly area,[Other],2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
7,8,232257,concern of loud noise in the Cheverly area,[Other],2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
8,9,232253,Complainant states odors from Stahl Point Road...,[Air],2024-03-07,2024-03-07,Anne Arundel,NaT,Under Investigation,NaN
9,10,232252,Complainant states odor from the Curtis Medica...,[Air],2024-03-07,2024-03-06,Anne Arundel,NaT,Under Investigation,NaN


In [41]:
# Now, see what it looks like when we run "explode" on these five rows.
last_year_df[0:19].explode("complaint_type")

,sno,complaint,complaint_description,complaint_type,recieved_date,incident_date,county,incident_closed_date,incident_status_desc,incident_zip
0,1,232264,concern of loud noise in the Cheverly area,Other,2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
1,2,232263,concern of loud noise in the Cheverly area,Other,2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
2,3,232262,concern of loud noise in the Cheverly area,Other,2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
3,4,232261,concern of loud noise in the Cheverly area,Other,2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
4,5,232260,concern of loud noise in the Cheverly area,Other,2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
5,6,232259,concern of loud noise in the Cheverly area,Other,2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
6,7,232258,concern of loud noise in the Cheverly area,Other,2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
7,8,232257,concern of loud noise in the Cheverly area,Other,2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
8,9,232253,Complainant states odors from Stahl Point Road...,Air,2024-03-07,2024-03-07,Anne Arundel,NaT,Under Investigation,NaN
9,10,232252,Complainant states odor from the Curtis Medica...,Air,2024-03-07,2024-03-06,Anne Arundel,NaT,Under Investigation,NaN


In [45]:
complaints_by_type = last_year_df.explode('complaint_type')

In [46]:
complaints_by_type['complaint_type'].value_counts()

complaint_type
Air                                   223
Odor                                  186
Other                                 126
Smoke                                  56
Fugitive Dust/Particulate Matter       45
Fumes                                  39
Open Burning                           30
Asbestos Complaint                      3
Air Pollutant Release                   2
Noise Complaint                         2
Non-Tidal Wetlands/Waterway             2
Suspected Operation without Permit      1
ARA AQCP non-regulated entity           1
Name: count, dtype: int64

In [47]:
last_year_by_type = complaints_by_type['complaint_type'].value_counts()

## Removing values from county 

In [48]:
last_year_df['county'].value_counts()

county
Baltimore City        101
Anne Arundel           85
Prince George's        83
Frederick              62
Baltimore              57
Montgomery             35
Cecil                  25
Dorchester             17
Harford                17
Howard                 15
Allegany               15
Washington             14
Charles                12
Carroll                 9
Wicomico                8
Not Yet Determined      7
Worcester               7
St. Mary's              6
Garrett                 5
Caroline                3
Queen Anne's            2
Somerset                2
Talbot                  1
Kent                    1
Statewide               1
Name: count, dtype: int64

In [50]:
invalid_county_rows = last_year_df[last_year_df['county'].isin(['Not Yet Determined', 'Statewide'])]
invalid_county_rows.to_csv('exported_data/invalid_counties.csv')

In [51]:
last_year_df = last_year_df[last_year_df['county'].isin(['Not Yet Determined', 'Statewide']) == False]

In [53]:
last_year_by_county = last_year_df['county'].value_counts()
last_year_by_county

county
Baltimore City     101
Anne Arundel        85
Prince George's     83
Frederick           62
Baltimore           57
Montgomery          35
Cecil               25
Harford             17
Dorchester          17
Howard              15
Allegany            15
Washington          14
Charles             12
Carroll              9
Wicomico             8
Worcester            7
St. Mary's           6
Garrett              5
Caroline             3
Queen Anne's         2
Somerset             2
Talbot               1
Kent                 1
Name: count, dtype: int64

# export data

In [56]:
last_year_by_type.to_csv('exported_data/complaint_type_frequency.csv')

last_year_by_county.to_csv('exported_data/country_frequency.csv')


In [57]:
last_year_df.to_csv('exported_data/all_complaints.csv', index=False)